### Using an ensemble of piven regressors

In [ ]:
n_regressors = 10
n = test_y.shape[0]
y_pred_out = np.zeros((n, n_regressors))
y_pred_pi_low = np.zeros((n, n_regressors))
y_pred_pi_high = np.zeros((n, n_regressors))

In [ ]:
# Convenience function
def make_model():
    # Put model in pipeline
    model = PivenKerasRegressor(build_fn=piven_model, 
                              input_dim=train_x.shape[-1], 
                              dense_units=(32,16), 
                              dropout_rate=(0.0,0.09),
                              lambda_=22.28,
                              lr= 0.000428)
    pipeline = Pipeline([
        ("preprocess", StandardScaler()),
        ("model", model)
    ])
    # Finally, normalize the output target
    model_ttr = PivenTransformedTargetRegressor(
        regressor=pipeline,
        transformer=StandardScaler()
    )
    return model_ttr

# Back-transform the predictions and PIs
def back_transform(x, max_train):
    xexp = np.exp(x)
    return max_train + 1 - xexp

In [ ]:
# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=2, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.00001
)
# Fit
for i in range(n_regressors):
    print(f"Fitting model {i+1}")  
    model_ttr = make_model()
    h = model_ttr.fit(train_x, train_y_transformed, model__epochs=25, 
                      model__validation_split=0.2, model__batch_size=64,
                      model__callbacks=[early_stop, reduce_lr])
    ypred, y_pi_low, y_pi_high = model_ttr.predict(test_x, 
                                                   return_prediction_intervals=True)

    y_pred_out[:,i] = back_transform(ypred, max_out_train)
    # Need to reverse the bounds
    y_pred_pi_low[:,i] = back_transform(y_pi_high, max_out_train)
    y_pred_pi_high[:,i] = back_transform(y_pi_low, max_out_train)

In [ ]:
np.save("y_pred.npy", y_pred_out)
np.save("y_pred_pi_low.npy", y_pred_pi_low)
np.save("y_pred_pi_high.npy", y_pred_pi_high)

In [ ]:
y_pred = np.mean(y_pred_out, axis=1)
y_pi_low = np.mean(y_pred_pi_low, axis=1)
y_pi_high = np.mean(y_pred_pi_high, axis=1)

In [ ]:
idx_sort = np.argsort(test_y)
plt.fill_betweenx(
    test_y[idx_sort],
    y_pi_low[idx_sort],
    y_pi_high[idx_sort],
    alpha=0.5
)
plt.scatter(y_pred[idx_sort], test_y[idx_sort], c="r", alpha=0.2)
plt.xlim(1900, 2015)
plt.title("Predicted versus true release years")
plt.xlabel("Predicted release year")
plt.ylabel("True release year")
plt.show()

In [ ]:
idx_sort = np.argsort(test_y)
for idx in range(y_pred_out.shape[1]):
    plt.plot(y_pred_pi_low[idx_sort, idx], test_y[idx_sort], alpha=0.6)
    plt.plot(y_pred_pi_high[idx_sort, idx], test_y[idx_sort], alpha=0.6)
plt.xlim(1860, 2015)
plt.title("Predicted versus true release years")
plt.xlabel("Predicted release year")
plt.ylabel("True release year")
plt.show()